In [ ]:
#!unzip -q /content/Mercedes_Logo_dataset.zip -d /content/
import os
import cv2
import shutil
import albumentations as A
from tqdm import tqdm

# === ШЛЯХИ ===
base_dir = r"Mercedes_Logo_dataset"
img_dir = os.path.join(base_dir, "train", "images")
label_dir = os.path.join(base_dir, "train", "labels")
aug_img_dir = os.path.join(base_dir, "train_aug", "images")
aug_label_dir = os.path.join(base_dir, "train_aug", "labels")
combined_img_dir = os.path.join(base_dir, "train_combined", "images")
combined_label_dir = os.path.join(base_dir, "train_combined", "labels")

# === СТВОРЕННЯ ДИРЕКТОРІЙ ===
os.makedirs(aug_img_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)
os.makedirs(combined_img_dir, exist_ok=True)
os.makedirs(combined_label_dir, exist_ok=True)

# === АУГМЕНТАЦІЇ ===
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.MotionBlur(p=0.3),
    A.GaussNoise(p=0.3),
    A.ColorJitter(p=0.3), #зміни кольору
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def load_yolo_labels(label_path):
    with open(label_path, 'r') as f:
        boxes, labels = [], []
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, x, y, w, h = map(float, parts)
                boxes.append([x, y, w, h])
                labels.append(int(cls))
    return boxes, labels

def save_yolo_labels(label_path, boxes, labels):
    with open(label_path, 'w') as f:
        for box, cls in zip(boxes, labels):
            f.write(f"{cls} {' '.join(f'{v:.6f}' for v in box)}\n")

NUM_AUGS = 3

# === АУГМЕНТАЦІЯ ЗОБРАЖЕНЬ ===
for img_name in tqdm(os.listdir(img_dir), desc="Аугментація"):
    if not img_name.lower().endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(img_dir, img_name)
    label_path = os.path.join(label_dir, os.path.splitext(img_name)[0] + ".txt")

    image = cv2.imread(img_path)
    if image is None or not os.path.exists(label_path):
        continue

    boxes, labels = load_yolo_labels(label_path)

    for i in range(NUM_AUGS):
        augmented = transform(image=image, bboxes=boxes, class_labels=labels)
        aug_image = augmented["image"]
        aug_bboxes = augmented["bboxes"]
        aug_labels = augmented["class_labels"]

        if not aug_bboxes:
            continue

        aug_img_name = os.path.splitext(img_name)[0] + f"_aug_{i}.jpg"
        aug_img_path = os.path.join(aug_img_dir, aug_img_name)
        aug_label_path = os.path.join(aug_label_dir, os.path.splitext(aug_img_name)[0] + ".txt")

        cv2.imwrite(aug_img_path, aug_image)
        save_yolo_labels(aug_label_path, aug_bboxes, aug_labels)

# === ОБ'ЄДНАННЯ ОРИГІНАЛ + АУГМЕНТОВАНІ ===
def copy_all(src_folder, dst_folder):
    for file_name in os.listdir(src_folder):
        src_path = os.path.join(src_folder, file_name)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.copy2(src_path, dst_path)

print("📦 Об’єднуємо train + train_aug → train_combined ...")
copy_all(img_dir, combined_img_dir)
copy_all(aug_img_dir, combined_img_dir)
copy_all(label_dir, combined_label_dir)
copy_all(aug_label_dir, combined_label_dir)
print("✅ Готово: train_combined створено.")


Аугментація: 100%|██████████| 280/280 [00:13<00:00, 20.43it/s]


📦 Об’єднуємо train + train_aug → train_combined ...
✅ Готово: train_combined створено.


In [ ]:
#!pip install ultralytics
from ultralytics import YOLO

# Завантаження моделі
model = YOLO("yolo11n.pt")

# Навчання моделі
model.train(
    data="Mercedes_Logo_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=32,
    name="mercedes_yolo_model",
    save=True
)

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Mercedes_Logo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mercedes_yolo_model5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pos

train: Scanning /content/Mercedes_Logo_dataset/train_combined/labels.cache... 1117 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1117/1117 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 372.9±114.5 MB/s, size: 49.5 KB)


val: Scanning /content/Mercedes_Logo_dataset/valid/labels.cache... 95 images, 0 backgrounds, 0 corrupt: 100%|██████████| 95/95 [00:00<?, ?it/s]


...

                   all         95        119       0.92      0.874      0.945      0.643



50 epochs completed in 0.286 hours.
Optimizer stripped from runs/detect/mercedes_yolo_model5/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/mercedes_yolo_model5/weights/best.pt, 5.5MB

Validating runs/detect/mercedes_yolo_model5/weights/best.pt...
Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


                   all         95        119       0.92      0.874      0.944      0.644
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/mercedes_yolo_model5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f1cd0f6d10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
# Встановлення необхідного
#!pip install ultralytics opencv-python --quiet

import cv2
from ultralytics import YOLO
from google.colab import files

# === ШЛЯХИ ===
model_path = r"runs/detect/mercedes_yolo_model5/weights/last.pt"
input_video_path = r"mercedes_converted.mp4"
output_video_path = r"output_50epoch_last_1920x1080_mercedes_video.mp4"

# === ЗАВАНТАЖЕННЯ МОДЕЛІ ===
model = YOLO(model_path)

# Відкриття відео
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"FPS: {fps}, Width: {width}, Height: {height}")

# Розмір для детекції
inference_size = (1920, 1080)

# Підготовка до запису відео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = 0

print("Обробка відео...")

while cap.isOpened():
    success, frame = cap.read()
    if not success or frame is None:
        break

    frame_count += 1
    #print(f"Обробка кадру {frame_count}")

    # Зменшення розміру кадру для моделі
    resized = cv2.resize(frame, inference_size)

    # Прогноз
    results = model(resized, verbose=False)[0]

    # Масштабування координат назад до розміру оригінального кадру
    scale_x = width / inference_size[0]
    scale_y = height / inference_size[1]

    for box in results.boxes:
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x1 = int(x1 * scale_x)
        y1 = int(y1 * scale_y)
        x2 = int(x2 * scale_x)
        y2 = int(y2 * scale_y)

        label = f"Logo {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Запис кадру
    out.write(frame)

# Завершення
cap.release()
out.release()
print(f"Готово. Збережено у файл: {output_video_path}")


FPS: 25, Width: 2560, Height: 1280
Обробка відео...
Готово. Збережено у файл: output_50epoch_last_1920x1080_mercedes_video.mp4


In [ ]:
!ffmpeg -i mercedes_video.mp4 -vcodec libx264 -acodec aac -strict experimental mercedes_converted.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import cv2
from ultralytics import YOLO

# === ШЛЯХИ ===
model_path = r"runs/detect/mercedes_yolo_model5/weights/best.pt"
input_video_path = r"mercedes_converted.mp4"
output_video_path = r"output_50epoch_NoRescale_mercedes_video.mp4"

# === ЗАВАНТАЖЕННЯ МОДЕЛІ ===
model = YOLO(model_path)

# Відкриття відео
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"FPS: {fps}, Width: {width}, Height: {height}")

# Підготовка до запису відео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = 0

print("Обробка відео...")

while cap.isOpened():
    success, frame = cap.read()
    if not success or frame is None:
        break

    # Прогноз на повнорозмірному кадрі
    results = model(frame, verbose=False)[0]

    for box in results.boxes:
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        label = f"Logo {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Запис кадру
    out.write(frame)

# Завершення
cap.release()
out.release()
print(f"Готово. Збережено у файл: {output_video_path}")

# Запропонувати завантаження
from google.colab import files
import os
if os.path.exists(output_video_path):
    print("Пропонуємо завантажити оброблене відео...")
    files.download(output_video_path)
else:
    print("Файл не знайдено для завантаження.")



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/ /content/drive/MyDrive/NeuroMKR/

Mounted at /content/drive
cp: cannot create directory '/content/drive/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/NeuroMKR/drive/.Encrypted/MyDrive/Neur